# Data Analysis

In [18]:
# Imports 
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [19]:
# Load the data
data = pd.read_csv('../data/owid-co2-data.csv')
data = data.dropna(subset=['co2'])

In [20]:
# Load total_ghg-emissions data
total_ghg_data = pd.read_csv('../data/total-ghg-emissions.csv')
total_ghg_data

,Entity,Code,Year,Annual greenhouse gas emissions in CO₂ equivalents
0,Afghanistan,AFG,1850,7338819.0
1,Afghanistan,AFG,1851,7403982.5
2,Afghanistan,AFG,1852,7464157.0
3,Afghanistan,AFG,1853,7522431.5
4,Afghanistan,AFG,1854,7579209.5
...,...,...,...,...
37190,Zimbabwe,ZWE,2018,37847984.0
37191,Zimbabwe,ZWE,2019,36351330.0
37192,Zimbabwe,ZWE,2020,33081062.0
37193,Zimbabwe,ZWE,2021,33889200.0


In [21]:
# Find correlation between columns
numeric_df = data.select_dtypes(include='number')
correlation = numeric_df.corr()
numeric_df.head()

,year,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,co2_including_luc,co2_including_luc_growth_abs,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
99,1949,7356890.0,NaN,0.0,0.0,0.015,NaN,NaN,6.268,NaN,...,NaN,0.125,0.0,0.0,0.000,0.0,NaN,NaN,NaN,NaN
100,1950,7480464.0,9.421400e+09,0.0,0.0,0.084,0.070,475.000,7.370,1.102,...,NaN,0.125,0.0,0.0,0.001,0.0,NaN,NaN,NaN,NaN
101,1951,7571542.0,9.692280e+09,0.0,0.0,0.092,0.007,8.696,8.232,0.861,...,NaN,0.125,0.0,0.0,0.001,0.0,NaN,NaN,NaN,NaN
102,1952,7667534.0,1.001733e+10,0.0,0.0,0.092,0.000,0.000,9.183,0.951,...,NaN,0.125,0.0,0.0,0.001,0.0,NaN,NaN,NaN,NaN
103,1953,7764549.0,1.063052e+10,0.0,0.0,0.106,0.015,16.000,10.256,1.073,...,NaN,0.125,0.0,0.0,0.001,0.0,NaN,NaN,NaN,NaN


In [22]:
# Display the correlation matrix
correlation_matrix = numeric_df.corr()

columns_of_interest = ['co2', 'gdp', 'population', 'cumulative_co2']  
correlation_matrix_filtered = numeric_df[columns_of_interest].corr()

fig = px.imshow(correlation_matrix_filtered, text_auto=True, labels=dict(color='Correlation'))
fig.update_layout(title='Filtered Correlation Matrix Heatmap', width=800, height=600)
fig.update_xaxes(tickangle=45)
fig.show()

In [23]:
# Select relevant columns for correlation
correlation_data = data[['gdp', 'population', 'co2', 'cumulative_co2', "total_ghg"]]
correlation_data = correlation_data.dropna()
correlation_matrix = correlation_data.corr()
correlation_matrix

,gdp,population,co2,cumulative_co2,total_ghg
gdp,1.000000,0.938576,0.977784,0.977087,0.981821
population,0.938576,1.000000,0.942418,0.887566,0.959142
co2,0.977784,0.942418,1.000000,0.964965,0.994183
cumulative_co2,0.977087,0.887566,0.964965,1.000000,0.961094
total_ghg,0.981821,0.959142,0.994183,0.961094,1.000000


In [24]:
# Select only numerical columns for correlation calculation
numerical_data = data.select_dtypes(include=['number'])
correlation_matrix_all_numeric = numerical_data.corr()
correlation_matrix_all_numeric

,year,population,gdp,cement_co2,cement_co2_per_capita,co2,co2_growth_abs,co2_growth_prct,co2_including_luc,co2_including_luc_growth_abs,...,share_global_other_co2,share_of_temperature_change_from_ghg,temperature_change_from_ch4,temperature_change_from_co2,temperature_change_from_ghg,temperature_change_from_n2o,total_ghg,total_ghg_excluding_lucf,trade_co2,trade_co2_share
year,1.000000,0.016694,0.077085,0.102981,0.312224,0.142780,0.046398,-0.009721,0.076471,0.010471,...,-0.726284,-0.144622,0.114512,0.104595,0.107659,0.124432,0.029292,0.031849,0.006132,0.065105
population,0.016694,1.000000,0.907211,0.802239,0.037464,0.842332,0.502810,-0.004865,0.908075,0.348481,...,0.590498,0.733955,0.938947,0.848382,0.882347,0.887376,0.945264,0.934036,-0.314998,-0.124726
gdp,0.077085,0.907211,1.000000,0.912824,0.077386,0.965254,0.311429,-0.002278,0.948413,0.171332,...,0.395960,0.673518,0.958481,0.959956,0.966770,0.976299,0.981821,0.980726,-0.070153,-0.056733
cement_co2,0.102981,0.802239,0.912824,1.000000,0.150730,0.890979,0.482679,-0.003283,0.853730,0.313276,...,0.405180,0.461275,0.853453,0.792499,0.818248,0.830563,0.917280,0.921456,-0.394426,-0.107764
cement_co2_per_capita,0.312224,0.037464,0.077386,0.150730,1.000000,0.141605,0.064110,-0.009446,0.096567,0.033119,...,-0.296477,-0.010834,0.089212,0.101050,0.099593,0.097290,0.091824,0.100188,-0.030735,-0.167459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
temperature_change_from_n2o,0.124432,0.887376,0.976299,0.830563,0.097290,0.962422,0.426666,-0.005000,0.953415,0.271134,...,0.502179,0.648452,0.970138,0.979878,0.987359,1.000000,0.977841,0.976756,-0.033170,-0.125654
total_ghg,0.029292,0.945264,0.981821,0.917280,0.091824,0.990723,0.465489,-0.012550,0.996540,0.295555,...,0.937712,0.972065,0.988881,0.967910,0.983109,0.977841,1.000000,0.997862,-0.182024,-0.117277
total_ghg_excluding_lucf,0.031849,0.934036,0.980726,0.921456,0.100188,0.996811,0.461929,-0.013050,0.996665,0.292195,...,0.943873,0.970249,0.981426,0.970729,0.983015,0.976756,0.997862,1.000000,-0.171555,-0.113502
trade_co2,0.006132,-0.314998,-0.070153,-0.394426,-0.030735,-0.130611,-0.357979,-0.038052,-0.180992,-0.227191,...,-0.259807,-0.053840,-0.207706,0.000514,-0.056494,-0.033170,-0.182024,-0.171555,1.000000,0.143160


In [25]:
# The co2 emissions ploted to show the primary_energy_consumption with plotly
fig = px.scatter(
    data,
    x='co2',
    y='primary_energy_consumption',
    title='CO2 Emissions vs Primary Energy Consumption',
    labels={'co2': 'CO2 Emissions', 'primary_energy_consumption': 'Primary Energy Consumption'}
)

fig.show()

## Predicting greenhouse emissions in the next 3 months

### Data Preparation

In [26]:
# Load Data
data = pd.read_csv('../data/owid-co2-data.csv')
data = data.dropna(subset=['co2'])

# Filter Data
clean_data = data.dropna(subset=["total_ghg", "year"]).copy()
clean_data['year'] = clean_data['year'].astype(int)
clean_data['Year'] = pd.to_datetime(clean_data['year'], format='%Y') + pd.offsets.YearEnd(0)
clean_data.set_index('Year', inplace=True)
clean_data['total_ghg'] = pd.to_numeric(clean_data['total_ghg'], errors='coerce')
clean_data = clean_data.dropna(subset=['total_ghg'])

# Resample Data to Annual_data
annual_data = clean_data['total_ghg'].resample('YE').sum()
annual_data.rename("Annual_GHG_Emissions", inplace=True)

print("Annual Data Sample:")
print(annual_data.head())

Annual Data Sample:
Year
1990-12-31    131590.571
1991-12-31    132244.173
1992-12-31    131955.451
1993-12-31    132495.800
1994-12-31    133533.677
Freq: YE-DEC, Name: Annual_GHG_Emissions, dtype: float64


### Train Test Split

In [27]:
train_size = int(len(annual_data) * 0.9)
train, test = annual_data[:train_size], annual_data[train_size:]
print(f"Training data shape: {train.shape}")
print(f"Testing data shape: {test.shape}")

years_to_predict = 4

Training data shape: (27,)
Testing data shape: (4,)


### ARIMA model

In [28]:
# ARIMA
p, d, q = 2, 1, 2
model_arima = ARIMA(train, order=(p, d, q))
model_arima_fit = model_arima.fit(method_kwargs={'maxiter': 200})

forecast_arima = model_arima_fit.forecast(steps=years_to_predict)
last_year_arima = annual_data.index[-1].year
forecast_index_arima = pd.date_range(start=f'{last_year_arima}-12-31', periods=len(test), freq='YE')

print("ARIMA Forecast:")
print(pd.Series(forecast_arima.values, index=forecast_index_arima))

# PLot ARIMA Forecast with plotly
fig_arima = go.Figure()

fig_arima.add_trace(go.Scatter(
    x=annual_data.index,
    y=annual_data,
    mode='lines',
    name='Actual Data',
    line=dict(color='red')
))

fig_arima.add_trace(go.Scatter(
    x=forecast_index_arima,
    y=forecast_arima,
    mode='lines',
    name='ARIMA Forecast',
    line=dict(color='blue', dash='dash')
))

fig_arima.update_layout(
    title='Total Greenhouse Gas Emissions Over Time with ARIMA Forecast',
    xaxis_title='Year',
    yaxis_title='Total Greenhouse Gas Emissions',
    legend=dict(x=0, y=1),
    hovermode='x unified'
)

fig_arima.show()    

ARIMA Forecast:
2020-12-31    190948.619813
2021-12-31    193605.465049
2022-12-31    195355.593620
2023-12-31    198011.244224
Freq: YE-DEC, dtype: float64


## Exponential Smoothing

In [35]:
# Exponential Smoothing v1
model_es = ExponentialSmoothing(
    annual_data,
    trend='add',
    seasonal=None,
    initialization_method='legacy-heuristic'
)

model_fit = model_es.fit()

last_year = annual_data.index[-1].year
forecast_index_es = pd.date_range(start=f'{last_year}', periods=years_to_predict, freq='YE')
forecast_es = model_fit.forecast(steps=years_to_predict)

fig_es = px.line(
    x=annual_data.index,
    y=annual_data,
    title="Total Greenhouse Gas Emissions Over Time with Exponential Smoothing Forecast",
    labels={'total_ghg': 'Emissions', 'Year': 'Year'}
)

fig_es.add_scatter(x=forecast_index_es, y=forecast_es, mode='lines', name='Forecast')
fig_es.show()

## Machine Learning Polynomial Regression

In [36]:
train_df = train.reset_index()

# Prepare training data
X_train = train_df['Year'].dt.year.values.reshape(-1, 1)
y_train = train_df['Annual_GHG_Emissions'].values

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('linear', LinearRegression())
])


param_grid = {
    'poly__degree': [1, 2, 3, 4, 5]  
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_degree = grid_search.best_params_['poly__degree']
best_model = grid_search.best_estimator_

print(f"Best polynomial degree found: {best_degree}")
print(f"Best cross-validation MSE: {-grid_search.best_score_:.4f}")

forecast_steps = len(test)
last_year = annual_data.index[-1].year
future_years = np.arange(last_year + 1, last_year + 1 + forecast_steps).reshape(-1, 1)


predictions = best_model.predict(future_years)
future_predictions = pd.DataFrame({
    'Year': future_years.flatten(),
    'Predicted_total_ghg': predictions
})

future_predictions['Year'] = pd.to_datetime(future_predictions['Year'], format='%Y')
future_predictions.set_index('Year', inplace=True)

print("Polynomial Regression Predictions:")
print(future_predictions)

# Plot Polynomial Regression Forecast with plotly
fig_poly = go.Figure()

fig_poly.add_trace(go.Scatter(
    x=annual_data.index,
    y=annual_data,
    mode='lines',
    name='Actual Data',
    line=dict(color='red')
))

fig_poly.add_trace(go.Scatter(
    x=future_predictions.index,
    y=future_predictions['Predicted_total_ghg'],
    mode='lines',
    name='Polynomial Regression Forecast',
    line=dict(color='green', dash='longdash')
))

fig_poly.update_layout(
    title='Total Greenhouse Gas Emissions Over Time with Polynomial Regression Forecast',
    xaxis_title='Year',
    yaxis_title='Total Greenhouse Gas Emissions',
    legend=dict(x=0, y=1),
    hovermode='x unified'
)

fig_poly.show()

Best polynomial degree found: 1
Best cross-validation MSE: 26640633.5244
Polynomial Regression Predictions:
            Predicted_total_ghg
Year                           
2021-01-01        202992.831233
2022-01-01        205530.542161
2023-01-01        208068.253090
2024-01-01        210605.964019


## Total Greenhouse Gas Emissions Over Time

In [31]:
# Load the total greenhouse gas emissions data 
total_ghg_data = total_ghg_data.dropna(subset=['Annual greenhouse gas emissions in CO₂ equivalents'])

total_ghg_data = total_ghg_data.groupby('Year').sum().reset_index()

# Plotly line Chart
fig = px.line(
    total_ghg_data,
    x='Year',
    y='Annual greenhouse gas emissions in CO₂ equivalents',
    title="Total Greenhouse Gas Emissions Over Time",
    labels={'total_ghg': 'Emissions', 'year': 'Year'}
)

fig.show()


## Comparing the predictions

In [37]:
# Plot the comparison
fig_combined = go.Figure()

# Actual Data
fig_combined.add_trace(go.Scatter(
    x=annual_data.index,
    y=annual_data.values,
    mode='lines',
    name='Actual Data',
    line=dict(color='red')
))

# ARIMA Forecast
fig_combined.add_trace(go.Scatter(
    x=forecast_index_arima,
    y=forecast_arima,
    mode='lines',
    name='ARIMA Forecast',
    line=dict(color='blue', dash='dash')
))

# Exponential Smoothing Forecast
fig_combined.add_trace(go.Scatter(
    x=forecast_index_es,
    y=forecast_es,
    mode='lines',
    name='Exponential Smoothing Forecast',
    line=dict(color='yellow', dash='dot')
))

# Polynomial Regression Forecast
fig_combined.add_trace(go.Scatter(
    x=future_predictions.index,
    y=future_predictions['Predicted_total_ghg'],
    mode='lines',
    name='Polynomial Regression Forecast',
    line=dict(color='green', dash='longdash')
))

fig_combined.update_layout(
    title='Total Greenhouse Gas Emissions Forecast Comparison',
    xaxis_title='Year',
    yaxis_title='Total Greenhouse Gas Emissions',
    legend=dict(x=0, y=1),
    hovermode='x unified'
)

fig_combined.show()

In [33]:
    # Calculate Errors
arima_mse = mean_squared_error(test, forecast_arima)

arima_mae = mean_absolute_error(test, forecast_arima)

es_mse = mean_squared_error(test, forecast_es)

es_mae = mean_absolute_error(test, forecast_es)

poly_mse = mean_squared_error(test, future_predictions['Predicted_total_ghg'])

poly_mae = mean_absolute_error(test, future_predictions['Predicted_total_ghg'])

print(f"ARIMA MSE: {arima_mse}")
print(f"ARIMA MAE: {arima_mae}")
print(f"Exponential Smoothing MSE: {es_mse}")

ARIMA MSE: 23623641.677693415
ARIMA MAE: 3735.8981854451267
Exponential Smoothing MSE: 23397248.299066477


In [34]:
# Make chart for errors
fig_errors = go.Figure()

fig_errors.add_trace(go.Bar(
    x=['ARIMA', 'Exponential Smoothing', 'Polynomial Regression'],
    y=[arima_mse, es_mse, poly_mse],
    name='Mean Squared Error',
    marker_color='blue'
))

fig_errors.add_trace(go.Bar(
    x=['ARIMA', 'Exponential Smoothing', 'Polynomial Regression'],
    y=[arima_mae, es_mae, poly_mae],
    name='Mean Absolute Error',
    marker_color='red'
))

fig_errors.update_layout(
    title='Model Error Comparison',
    xaxis_title='Model',
    yaxis_title='Error',
    barmode='group'
)

fig_errors.show()